In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Clone Transformers and Prepare env

In [ ]:
%cd /content

/content


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.1 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4307 sha256=20fa1e6a7da1ab7c4f8e2304583cbdbd74fa3cc354fe11b6cdf3a88bf5d60943
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


#Preprocessing Data

In [ ]:
def xoa_trung_lap(s):
  loop = ""
  le = len(s)
  i=1
  while i <= len(s)-1:
    if s[i]==s[i-1] and (i==len(s)-1 or s[i+1]==' '):
      j=i
      loop=s[i]
      while s[j-1] == s[j]:
        loop+=s[j]
        j-=1
      s = s.replace(loop, s[i])
    i+=1
  return s

print(xoa_trung_lap("thầy rất dễ thươngggggggggg đó là điều mà em không bao giờ nói  dạy học mà như dân chợ búaaaaa"))

thầy rất dễ thương đó là điều mà em không bao giờ nói  dạy học mà như dân chợ búa


In [ ]:
#word_segment
import py_vncorenlp

py_vncorenlp.download_model(save_dir='/content')

rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

output = rdrsegmenter.word_segment(text)

print(output)

['Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội .', 'Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .']


#Loading Model to Train

In [ ]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer
from nltk.tokenize.casual import TweetTokenizer

model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
list_emoji = pd.read_csv('/content/drive/MyDrive/CS114/emoji.csv', header=None)

In [ ]:
print(list_emoji[0])


0    :)
1    =)
2     😂
3     😊
4     ❤
5     🤗
6     😅
7     😢
Name: 0, dtype: object


In [ ]:
tokenizer.add_tokens(list(list_emoji[0]))

8

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(64009, 768)

In [ ]:
tokenizer.tokenize(':) hehe onana =) 😍 😅')

[':)', 'he@@', 'he', 'on@@', 'ana', '=)', '😍', '😅']

In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/CS114/train.csv')

In [ ]:
def process_data2(row):
  # sentence = xoa_trung_lap(row[0])
  sen_af = rdrsegmenter.word_segment(row[0])
  label = 0
  if row[1] == 'N':
    label = 0
  elif row[1] == 'P':
    label = 1
  else:
    label = 2
  return sen_af[0], label

In [ ]:
arr1 = []
arr2 = []
for i in train_df.values:
  x, y = process_data2(i)
  arr1.append(x)
  arr2.append(y)

In [ ]:
d={'text':arr1, 'label': arr2}
df_csv = pd.DataFrame(data=d)
df_csv[:5]

,text,label
0,Mu chỉ đủ trình đá C3 thôi .,0
1,"mỗi lần cảm_thấy mệt_mỏi , nhớ nhà liền nghe l...",1
2,Xuống c2 chơi gặp liver mân đàn khổ,0
3,Onana bàn thua thứ hai định đưa chân khiến thu...,2
4,En tô no Ma ti eo,2


In [ ]:
df_csv.to_csv("/content/drive/MyDrive/CS114/data/train.csv", index=False)
df_csv.to_csv("/content/drive/MyDrive/CS114/data/test.csv", index=False)

#Load data

In [ ]:
from datasets import load_dataset
data_files = {"train": "train.csv", "test": "test.csv"}
train_dataset = load_dataset("/content/drive/MyDrive/CS114/data", split="train")
val_dataset = load_dataset("/content/drive/MyDrive/CS114/data", split="test")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
print(val_dataset[2], train_dataset[2])

{'text': 'Xuống c2 chơi gặp liver mân đàn khổ', 'label': 0} {'text': 'Xuống c2 chơi gặp liver mân đàn khổ', 'label': 0}


#Traning

In [ ]:
from typing import List
from transformers import TrainingArguments, Trainer

In [ ]:
def tokenization(batched_text):
    tokenized_batch = tokenizer(batched_text['text'], padding = True, truncation=True)
    return tokenized_batch


train_dataset = train_dataset.map(tokenization, batched = True, batch_size = len(train_dataset))
val_dataset = val_dataset.map(tokenization, batched = True, batch_size = len(val_dataset))

Map:   0%|          | 0/1603 [00:00<?, ? examples/s]

Map:   0%|          | 0/1603 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
training_args = TrainingArguments(
  output_dir='./results',
  num_train_epochs=3,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=16,
  remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [ ]:
trainer.train()

Step,Training Loss
500,0.719300


TrainOutput(global_step=603, training_loss=0.6739794666296609, metrics={'train_runtime': 227.4474, 'train_samples_per_second': 21.143, 'train_steps_per_second': 2.651, 'total_flos': 632656212926976.0, 'train_loss': 0.6739794666296609, 'epoch': 3.0})

In [ ]:
device = torch.device("cuda")
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64009, 768)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
#predict thu 1 example
import numpy as np
ans=[]
sentence = 'Xuống c2 chơi gặp liver mân đàn khổ =))))'
be = sentence
# sentence = scanerr(sentence)
sen_af = rdrsegmenter.word_segment(sentence)
input_ids = torch.tensor([tokenizer.encode(sen_af[0])]).to(device)

with torch.no_grad():
    out = model(input_ids)
    if np.argmax(out.logits.softmax(dim=-1).tolist())==0:
      ans.append('negative')
    elif np.argmax(out.logits.softmax(dim=-1).tolist())==1:
      ans.append('positive')
    else:
      ans.append('neutral')
    print(be)
    print(tokenizer.tokenize(be))
    print(out.logits.softmax(dim=-1).tolist())

Xuống c2 chơi gặp liver mân đàn khổ =))))
['Xuống', 'c@@', '2', 'chơi', 'gặp', 'liver', 'm@@', 'ân', 'đàn', 'khổ', '=)', ')@@', ')@@', ')']
[[0.6734293699264526, 0.036110568791627884, 0.2904600501060486]]


In [ ]:
model.save_pretrained('/content/drive/MyDrive/CS114/no_emoji_model') #dir to the folder save model

#Evaluate

In [ ]:
#evaluate model vua train
trainer.evaluate()

{'eval_loss': 0.30468711256980896,
 'eval_accuracy': 0.9008109794135996,
 'eval_f1': 0.9008473583874572,
 'eval_precision': 0.9099215440183518,
 'eval_recall': 0.89475217599861,
 'eval_runtime': 22.3531,
 'eval_samples_per_second': 71.713,
 'eval_steps_per_second': 4.518,
 'epoch': 3.0}

In [ ]:
#load model tu file output vua luu
model1 = RobertaForSequenceClassification.from_pretrained("/content/out_model")
model1.cuda()

In [ ]:
#load model da duoc train truoc day
model2 = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/retrained")
model2.cuda()

In [ ]:
import numpy as np

In [ ]:
ans = []
ans1 = []
ans2 = []
eval = []
eval1 = []
eval2 = []

for sentence in val_dataset['text']:
  be = sentence
  sentence = scanerr(sentence)
  sen_af = rdrsegmenter.word_segment(sentence)
  input_ids = torch.tensor([tokenizer.encode(sen_af[0])]).to(device)

  with torch.no_grad():
      out = model(input_ids)
      if np.argmax(out.logits.softmax(dim=-1).tolist())==0:
        ans.append('negative')
      elif np.argmax(out.logits.softmax(dim=-1).tolist())==1:
        ans.append('positive')
      else:
        ans.append('neutral')
      eval.append(np.argmax(out.logits.softmax(dim=-1).tolist()))
      out1 = model1(input_ids)
      if np.argmax(out1.logits.softmax(dim=-1).tolist())==0:
        ans1.append('negative')
      elif np.argmax(out1.logits.softmax(dim=-1).tolist())==1:
        ans1.append('positive')
      else:
        ans1.append('neutral')
      eval1.append(np.argmax(out1.logits.softmax(dim=-1).tolist()))
      out2 = model2(input_ids)
      if np.argmax(out2.logits.softmax(dim=-1).tolist())==0:
        ans2.append('negative')
      elif np.argmax(out2.logits.softmax(dim=-1).tolist())==1:
        ans2.append('positive')
      else:
        ans2.append('neutral')
      eval2.append(np.argmax(out2.logits.softmax(dim=-1).tolist()))
      print(be)
      print(sentence)
      print(ans[-1])
      print(ans1[-1])
      print(ans2[-1])


tài_liệu được bổ_sung nhiều và đầy_đủ trên moodle .
tàiliệu được bổsung nhiều và đầyđủ trên moodle .
positive
positive
positive
giảng_viên phải nên cho ví_dụ cụ_thể những phần quan_trọng nên nói kỹ có_thể lấy ví_dụ minh_hoạ tránh việc lập đi lập lại nhiều lần sẽ tạo cảm_giác chán cho sinh_viên .
giảngviên phải nên cho vídụ cụthể những phần quantrọng nên nói kỹ cóthể lấy vídụ minhhoạ tránh việc lập đi lập lại nhiều lần sẽ tạo cảmgiác chán cho sinhviên .
negative
negative
negative
bài_tập đa_dạng , cụ_thể .
bàitập đadạng  cụthể .
negative
negative
negative
slide up đầy_đủ trước mỗi buổi học .
slide up đầyđủ trước mỗi buổi học .
positive
positive
neutral
nội_dung giảng_dạy quá nhàm_chán , khô_khan .
nộidung giảngdạy quá nhàmchán  khôkhan .
negative
negative
negative
thầy chép bảng nhiều .
thầy chép bảng nhiều .
negative
negative
neutral
thầy tận_tình hướng_dẫn cách làm , giải_quyết những thắc_mắc của sinh_viên .
thầy tậntình hướngdẫn cách làm  giảiquyết những thắcmắc của sinhviên .
positi

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(eval,val_dataset['label'], average='macro'))
print(f1_score(eval1,val_dataset['label'], average='macro'))
print(f1_score(eval2,val_dataset['label'], average='macro'))


0.6484509145203434
0.6850843584055926
0.7077414455463237


#Infer

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment") #day la default model, ban co the folder chua model khac

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/data_phase_2.txt", sep="\t", header=None)
print(test_df[0][:5])

0                     Tiến sĩ câu giờ, hoàng đế gây mê
1    Mới học 2 bữa thôi là đã nghiện cách giảng dạy...
2     kêu giơ tay phát biểu cộng điểm nhưng có cái nịt
3               thầy mãi là number one trong lòng em !
4    Thầy như là người được chọn, sỡ hữu haki bá vư...
Name: 0, dtype: object


In [ ]:
import numpy as np
ans = []
for sentence in test_df[0]:
  be = sentence
  sentence = scanerr(sentence)
  sen_af = rdrsegmenter.word_segment(sentence)
  input_ids = torch.tensor([tokenizer.encode(sen_af[0])])


  with torch.no_grad():
      out = model(input_ids)
      if np.argmax(out.logits.softmax(dim=-1).tolist())==0:
        ans.append('negative')
      elif np.argmax(out.logits.softmax(dim=-1).tolist())==1:
        ans.append('positive')
      else:
        ans.append('neutral')
      print(be)
      print(sentence)
      print(np.argmax(out.logits.softmax(dim=-1).tolist()))

Tiến sĩ câu giờ, hoàng đế gây mê
tiến sĩ câu giờ hoàng đế gây chán buồn ngủ
0
Mới học 2 bữa thôi là đã nghiện cách giảng dạy của thầy rồi
mới học 2 bữa thôi là đã nghiện cách giảng dạy của thầy rồi
1
kêu giơ tay phát biểu cộng điểm nhưng có cái nịt
kêu giơ tay phát biểu cộng điểm nhưng có cái nịt
0
thầy mãi là number one trong lòng em !
thầy mãi là number one trong lòng em 
1
Thầy như là người được chọn, sỡ hữu haki bá vương, mỗi khi thầy giảng bài cả lớp đều gục ngã
thầy như là người được chọn sỡ hữu haki bá vương mỗi khi thầy giảng bài cả lớp đều gục ngã
0
không hài lòng điểm gì cả !
không hài lòng điểm gì cả 
0
Hoàn toàn không có điểm nào không hài lòng cả.
hoàn toàn không có điểm nào không hài lòng cả.
0
Nếu một ngày, thầy nghe được từ em nói rằng thầy dạy hay quá, em yêu thầy ... thì chắc chắn hôm đó chỉ có thể là cá tháng tư
nếu một ngày thầy nghe được từ em nói rằng thầy dạy hay quá em yêu thầy . thì chắc chắn hôm đó chỉ có thể là cá tháng tư
1
Học với thầy, các bạn sẽ phải học 

In [ ]:
with open("/content/submit.txt", "w") as txt_file:
    for line in ans:
        txt_file.write("".join(line) + "\n")